### 객체를 제너릭하게 구성하려면 @classmethod를 통한 다양성을 활용하라
- 다형성을 사용하면 계층을 이루는 여러 클래스가 자신에게 맞는 유일한 메서드 버전을 구현할 수 있다.
- 이는 같은 인터페이스를 만족하거나 같은 추상 기반 클래스를 공유하는 많은 클래스가 서로 다른 기능을 제공할 수 있다는 뜻이다.
- 예를 들어 맵리듀스 구현을 하나 작성하고 있는데, 입력 데이터를 표현할 수 잇는 공통 클래스가 필요하다고 하자.
    - 아래 코드는 이럴 때 사용하기 위해 정의한, 하위 클래스에서 다시 정의해야만 하는 read 메서드가 들어있는 공통 클래스를 보여준다.
    

In [26]:
class InputData:
    def read(self):
        raise NotImplementedError
print(InputData.read)

<function InputData.read at 0x00000227FE7C7CA0>


- 이 inputData의 구체적인 하위 클래스를 만들면서 디스크에 파일을 읽게 할 수 있다.

In [27]:
class PathInputData(InputData):
    def __init__(self, path):
        super().__init__()
        self.path = path

    def read(self):
        with open(self.path) as f:
            return f.read()

- PathInputData와 같이 원하면 얼마든지 InputData의 하위 클래스를 만들 수 있다.
- 각 하위 클래스는 처리할 데이터를 돌려주는 공통 read 인터페이스를 구현해야한다.
- 어떤 InputData의 하위 클래스는 네트워크에서 데이터를 읽을 수 있다
- 또 다른 하위 클래스는 읽어온 압축된 데이터를 추명하게 풀어서 제공할 수 있다.

In [28]:
# 위와 비슷한 방법으로 이 입력 데이터를 소비하는 공통 방법을 제공하는 맵리듀스 작업자로 쓸수 있는 추상 인터페이스를 정의하고싶다.
class Worker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None

    def map(self):
        raise NotImplementedError

    def reduce(self, other):
        raise NotImplementedError

In [29]:
# 아래 코드는 맵리듀스 기능을 구현하는 Worker의 구체적인 하위 클래스다.
class LineCountWorker(Worker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n')
        

    def reduce(self, other):
        self.result += other.result

-  이 구현은 아주 잘 작동할 것 처럼 보인다.
- 하지만 모든 요소를 구현하는 과정에서 가장 큰 난관이 있다.
- 대체 각 부분을 어떻게 연결해야 할까?
- 이해하기 쉬운 인터페이스와 추상화를 제공하는 멋진 클래스를 여럿 만들었지만. 객체를 생성해 활용해야만 이 모든 클래스가 쓸모있게 된다.
- 각 객체를 만들고 맴리듀스를 조화롭게 실행하는 책임은 누가 져야할까?

[https://3months.tistory.com/521]

- 가장 간단한 접근 방법은 도우미 함수를 활용해 객체를 직접 만들고 연결한다.
- 다음 코드는 디렉토리의 목록을 얻어서 그 안에 들어 있는 파일마다 PathInputData 인스턴스를 만든다.

In [30]:
import os

def generate_inputs(data_dir):
    for name in os.listdir(data_dir):
        yield PathInputData(os.path.join(data_dir, name))


- 다음으로 방금 generate_inputs를 통해 만든 InputData 인스턴스들을 사용하는 LineCountWorker 인스턴스를 만든다.

In [31]:
def create_workers(input_list):
    workers = []
    for input_data in input_list:
        workers.append(LineCountWorker(input_data))
    return workers


- Worker 인스턴스의 map 단계를 여러 스레드에 공급해서 실행할 수 있다.
- 그 후 reduce를 반복적으로 호출해서 결과를 최종 값으로 합칠 수 있다.


In [32]:
from threading import Thread


def execute(workers):
    threads = [Thread(target=w.map) for w in workers]
    for thread in threads: thread.start()
    for thread in threads: thread.join()

    first, *rest = workers
    for worker in rest:
        first.reduce(worker)
    return first.result

- 마지막으로 지금까지 만든 모든 조각을 한 함수 안에 합쳐서 각 단계를 실행 한다.


In [33]:
def mapreduce(data_dir):
    inputs = generate_inputs(data_dir)
    workers = create_workers(inputs)
    return execute(workers)

- 몇 가지 입력 파일을 대상으로 이 함수를 실행해보면 아주 훌륭하게 작동한다.

In [36]:
import os
import random

def write_test_files(tmpdir):
    os.makedirs(tmpdir)
    for i in range(100):
        with open(os.path.join(tmpdir, str(i)), 'w') as f:
            f.write('\n' * random.randint(0, 100))

tmpdir = 'test_inputs'
write_test_files(tmpdir)

result = mapreduce(tmpdir)
print(f'총 {result} 줄이 있습니다.')

총 4708 줄이 있습니다.


- 앞에서 정의한 mapreduce 함수의 가장 큰 문제점은 함수가 전혀 제너릭하지 않다는 것이다.
- 다른 InputData나 Worker 하위 클래스를 사용하고 싶다면 각 하위 클래스에 맞게 generate_input,create_workers,mapreduce를 재작성해야한다.

##### 문제의 핵심
- 객체를 구성할 수 있는 제너릭한 방법이 필요하다는 점이다.
- 다른 언어에서는 다형성을 활용해 이 문제를 해결할 수 있다.
- inputData의 모든 하위 클래스는 맵리듀스를 처리하는 도우미 메서드들이 제너릭하게 사용할 수 있는 특별한 생성자를 제공한다.,
- 파이썬에서는 생성자 메서드가 $__init__$ 밖에 없다는 점이 문제다.
- InputData의 모든 하위 클래스가 똑같은 생성자만 제공해야 한다고 요구하는 것은 불합리하다.

##### 문제를 해결하는 좋은 방법은 *클래스 메서드* 다형성을 사용하는 것이다.
- 이 방식은 InputData.read에서 사용했던 인스턴스 메서드의 다형성과 똑같다.
- 클래스로 만들어낸 개별 객체에 적용되지 않고 클래스 전체에 적용된다는 점만 다르다.


##### 클래스 메서드라는 아이디어를 맵리듀스에 사용했던 클래스에 적용해보자.
- 아래 코드는 inputData에 제너릭 @classmethod를 적용한 모습이다.
- @classmethod가 적용된 클래스 메서드는 공통 인터페이스를 통해 새로운 InputData 인스턴스를 생성한다.

In [38]:
class GenericInputData:
    def read(self):
        raise NotImplementedError

    @classmethod
    def generate_inputs(cls, config):
        raise NotImplementedError

- generate_inputs는 GenericInputData의 구체적인 하위 클래스가 객체를 생성하는 방법을 알려주는 설정 정보가 들어있는 딕셔너리를 파라미터로 받는다.
- 다음 코드는 입력 파일이 들어있는 디렉토리를 찾기 위해 이 config를 사용한다.

In [39]:

class PathInputData(GenericInputData):
    def __init__(self, path):
        super().__init__()
        self.path = path

    def read(self):
        with open(self.path) as f:
            return f.read()

    @classmethod
    def generate_inputs(cls, config):
        data_dir = config['data_dir']
        for name in os.listdir(data_dir):
            yield cls(os.path.join(data_dir, name))#cls로 클래스 속성에 접근(genericinputData)


- 비슷한 방식으로 GenericWorker 클래스 안에 creat_workers 도우미 메서드를 추가할 수 있다.
- 도우미 메서드는 GenericInputData의 하위 타입이어야 하는 input_class를 파라미터로 받는다.
- input_class는 필요한 입력을 생성한다.
- GenericWorker의 구체적인 하위 타입의 인스턴스를 만들 때는 cls()를 제너릭생성자로 사용한다.

In [41]:
class GenericWorker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None

    def map(self):
        raise NotImplementedError

    def reduce(self, other):
        raise NotImplementedError

    @classmethod
    def create_workers(cls, input_class, config):
        workers = []
        for input_data in input_class.generate_inputs(config):
            workers.append(cls(input_data))
        return workers


- 이 코드에서 input_class.generate_inputs 호출이 바로 여기서 보여주려는 클래스 다형성의 예다.
- create_workers가 __init__ 메서드를 직접 호출하지 않고 cls()를 호출함으로써 다른 방법으로 GenericWorker 객체를 만들수 있다는 것도 알 수 있다.
- 이런 변경이 구체적인 GenericWorker 하위 클래스에 미치는 영향은 부모 클래스를 바꾸는 것뿐이다.

In [43]:
class LineCountWorker(GenericWorker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n')

    def reduce(self, other):
        self.result += other.result



- 마지막으로 mapreduce 함수가 create_workers를 호출하게 변경해서 mapreduce를 완전한 제너릭 함수로 만들 수 있다.

In [46]:
def mapreduce(worker_class, input_class, config):
    workers = worker_class.create_workers(input_class, config)
    return execute(workers)

- 똑같은 테스트 파일 집합에 대해 새로운 작업자를 실행하면 이전의 구현과 똑같은 구현을 얻을 수 있다.
- 유일한 차이점은 제너릭하게 작동해야 하므로 mapreduce 함수에 더 많은 파라미터를 넘겨야 한다는 것뿐이다.

In [47]:
config = {'data_dir': tmpdir}
result = mapreduce(LineCountWorker, PathInputData, config)
print(f'총 {result} 줄이 있습니다.')

[<__main__.LineCountWorker object at 0x00000227FE7CA6D0>, <__main__.LineCountWorker object at 0x00000227FE7CA5B0>, <__main__.LineCountWorker object at 0x00000227FE7CAE80>, <__main__.LineCountWorker object at 0x00000227FE7CA6A0>, <__main__.LineCountWorker object at 0x00000227FE7CA760>, <__main__.LineCountWorker object at 0x00000227FE71DCD0>, <__main__.LineCountWorker object at 0x00000227FE71D040>, <__main__.LineCountWorker object at 0x00000227FE71D6A0>, <__main__.LineCountWorker object at 0x00000227FE71D3D0>, <__main__.LineCountWorker object at 0x00000227FE6D5AF0>, <__main__.LineCountWorker object at 0x00000227FE6D5A90>, <__main__.LineCountWorker object at 0x00000227FE6D5610>, <__main__.LineCountWorker object at 0x00000227FE6E8400>, <__main__.LineCountWorker object at 0x00000227FE6E8370>, <__main__.LineCountWorker object at 0x00000227FE7FB730>, <__main__.LineCountWorker object at 0x00000227FE7FB520>, <__main__.LineCountWorker object at 0x00000227FE7FBB20>, <__main__.LineCountWorker obje

- 각 하위 클래스의 인스턴스 객체를 결합하는 코드를 변경하지 않아도 GenericInputData와 GenericWorker의 하위 클래스를 내가 원하는 대로 작성 할 수 있다.

[https://wikidocs.net/16074]